In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the LabeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = wg.LabeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 
         'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical','decriminalization':'categorical', 
             'medical':'categorical', 'recreational':'categorical', 'no_reforms':False,}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops',
             'contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.meta_df['role'][k] = v
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_intersectional(['driver_gender','driver_race'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0,F_White
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0,M_White
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian


In [11]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0,F_White
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0,M_White
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian


In [12]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


In [13]:
labeled_df.add_quantile(['hit_rate','num_stops'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,31.0,NaN,NaN,1.0,NaN,NaN,31.0,F_White,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,2.0,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,15.0,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,34.0,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,61.0,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,high,low


In [14]:
labeled_df.get_vars_per_type('categorical')

['state',
 'driver_gender',
 'driver_race',
 'decriminalization',
 'medical',
 'recreational',
 'driver_gender_driver_race',
 'hit_ratequantiles',
 'num_stopsquantiles']

In [15]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [16]:
corrobj = wg.All_Pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [17]:
rankobj = wg.Mean_Rank_Trend()
linreg_obj = wg.All_Linear_Trend()

# Computing Trends on a LabeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [18]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

replacing True False


,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,agg_trend,agg_trend_strength
0,year,search_conducted_rate,-0.247018,0.247018,state,AZ,pearson_corr,-0.03903,0.03903
1,year,search_conducted_rate,-0.414566,0.414566,state,CO,pearson_corr,-0.03903,0.03903
2,year,search_conducted_rate,0.118238,0.118238,state,CT,pearson_corr,-0.03903,0.03903
3,year,search_conducted_rate,-0.199765,0.199765,state,IL,pearson_corr,-0.03903,0.03903
4,year,search_conducted_rate,-0.603026,0.603026,state,MA,pearson_corr,-0.03903,0.03903


Now we can use a list of objects and apply multiple trends

In [19]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/trend_components/regression.py:87: UserWarning: cannot compute with two different weights
  warnings.warn('cannot compute with two different weights')


replacing True False


,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,agg_trend,agg_trend_strength
19,year,search_conducted_rate,-0.115382,0.115382,driver_race,White,pearson_corr,-0.0390297,0.039030
875,num_stops,state,"[VT, RI, IL, CT, WI, CO, MD, TX, MA, SC, WA, N...",0.745900,driver_gender_driver_race,F_Other,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900
147,year,num_stops,0.104919,0.104919,driver_race,Asian,pearson_corr,-0.043965,0.043965
31,year,search_conducted_rate,0.137845,0.137845,driver_gender_driver_race,F_Other,pearson_corr,-0.0390297,0.039030
710,num_stops,driver_gender,[F],NaN,driver_gender_driver_race,F_Black,rank_trend,"[F, M]",0.116400
140,year,num_stops,-0.0394452,0.039445,state,SC,pearson_corr,-0.043965,0.043965
459,search_conducted_rate,driver_gender,"[F, M]",0.399300,decriminalization,1,rank_trend,"[F, M]",0.251900
150,year,num_stops,0.0751798,0.075180,driver_race,Other,pearson_corr,-0.043965,0.043965
761,num_stops,driver_race,"[Black, Hispanic, White]",0.289000,num_stopsquantiles,high,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.499600
286,search_conducted_rate,num_stops,-0.0569225,0.056923,medical,0,pearson_corr,-0.0655937,0.065594


In [20]:
replace = False
labeled_df.result_df.empty  or replace

False

In [21]:
labeled_df.get_pairwise_trends_1lev([rankobj,linreg_obj])


,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,subgroup_trend2,subgroup_trend_strength2,subgroup2
0,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.0128229,0.012823,CO
1,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,-0.0367598,0.036760,CT
2,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.380135,0.380135,IL
3,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.405733,0.405733,MA
4,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.148585,0.148585,MD
5,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.277856,0.277856,NC
6,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.130845,0.130845,RI
7,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.122165,0.122165,SC
8,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.313004,0.313004,TX
9,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.0888378,0.088838,VT


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [22]:
pd.unique(labeled_df.result_df['trend_type'])

array(['pearson_corr', 'rank_trend', 'lin_reg'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [23]:
labeled_df.trend_list

In [24]:
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,subgroup_trend2,subgroup_trend_strength2,subgroup2
0,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.0128229,0.012823,CO
1,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,-0.0367598,0.036760,CT
2,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.380135,0.380135,IL
3,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.405733,0.405733,MA
4,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,0.148585,0.148585,MD


In [25]:
# a = ['a','b','c','d']
# b = ['a','b','d','e']
a = ['F', 'M']
b = ['F', 'M']
for i, d in enumerate(set(a+b)):
    print (i,':',d)

0 : F
1 : M


In [26]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance('subgroup_trend','subgroup_trend2')

labeled_df.result_df.sample(10)

,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,subgroup_trend2,subgroup_trend_strength2,subgroup2,distance
27,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.130845,0.130845,RI,1.0000
4432,hit_rate,driver_gender,"[F, M]",0.180600,state,CT,rank_trend,"[F, M]",0.156200,MA,0.0000
1971,search_conducted_rate,driver_gender,"[F, M]",0.415300,state,RI,rank_trend,"[F, M]",0.636000,SC,0.0000
4984,num_stops,state,"[AZ, RI, WI, MA, IL, NC, SC, TX]",0.289600,medical,0,rank_trend,"[RI, VT, CT, MD, IL, CO, AZ, MA, WA]",0.340200,1,0.3333
536,year,contraband_found_rate,-0.0599275,0.059928,state,SC,pearson_corr,-0.0223724,0.022372,TX,0.0000
198,hit_rate,num_stops,0.390729,0.390729,state,MA,pearson_corr,0.0152053,0.015205,MD,0.0000
1068,search_conducted_rate,contraband_found_rate,0.230634,0.230634,driver_gender_driver_race,F_Asian,pearson_corr,0.576803,0.576803,F_White,0.0000
4600,num_stops,driver_gender,"[F, M]",0.200000,state,IL,rank_trend,"[F, M]",0.091600,RI,0.0000
756,year,search_conducted_rate,-0.386635,0.386635,state,MD,pearson_corr,-0.120095,0.120095,VT,0.0000
654,year,num_stops,-0.0517808,0.051781,state,CT,pearson_corr,0.0444106,0.044411,VT,1.0000


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [27]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':                            year  search_conducted_rate  contraband_found_rate  \
 year                   1.000000              -0.039030               0.041129   
 search_conducted_rate -0.039030               1.000000               0.807915   
 contraband_found_rate  0.041129               0.807915               1.000000   
 hit_rate               0.316196              -0.094273               0.275397   
 num_stops             -0.043965              -0.065594              -0.073456   
 
                        hit_rate  num_stops  
 year                   0.316196  -0.043965  
 search_conducted_rate -0.094273  -0.065594  
 contraband_found_rate  0.275397  -0.073456  
 hit_rate               1.000000  -0.042102  
 num_stops             -0.042102   1.000000  ,
 'pearson_corr_subgroup_trend':                                               year  search_conducted_rate  \
 num_stopsquantiles                                                          
 high           

# Filtering

In [28]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module wiggum.ranking_processing:

get_trend_rows(feat1=None, feat2=None, group_feat=None, subgroup=None, trend_type=None) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    feat1 : str, list, or  None
        trend variable name or None to include all
    feat2 : str, list, or  None
        trend variable name or None to include all
    group_feat : str, list, or  None
        groupoby variable name or None to include all
    subgroup : str, list, or  None
        value of groupby_feat or or None to include all



So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [29]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

40  total rows meet the criteria


,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,subgroup_trend2,subgroup_trend_strength2,subgroup2,distance
854,year,contraband_found_rate,0.0408099,0.040810,driver_race,Black,pearson_corr,0.0425516,0.042552,Hispanic,0.000000
855,year,contraband_found_rate,0.0408099,0.040810,driver_race,Black,pearson_corr,0.114371,0.114371,Other,0.000000
856,year,contraband_found_rate,0.0408099,0.040810,driver_race,Black,pearson_corr,-0.0349623,0.034962,White,1.000000
857,year,contraband_found_rate,0.0425516,0.042552,driver_race,Hispanic,pearson_corr,0.114371,0.114371,Other,0.000000
858,year,contraband_found_rate,0.0425516,0.042552,driver_race,Hispanic,pearson_corr,-0.0349623,0.034962,White,1.000000
864,year,hit_rate,0.354374,0.354374,driver_race,Black,pearson_corr,0.329214,0.329214,Hispanic,0.000000
865,year,hit_rate,0.354374,0.354374,driver_race,Black,pearson_corr,0.243678,0.243678,Other,0.000000
866,year,hit_rate,0.354374,0.354374,driver_race,Black,pearson_corr,0.355338,0.355338,White,0.000000
867,year,hit_rate,0.329214,0.329214,driver_race,Hispanic,pearson_corr,0.243678,0.243678,Other,0.000000
868,year,hit_rate,0.329214,0.329214,driver_race,Hispanic,pearson_corr,0.355338,0.355338,White,0.000000


In [30]:
labeled_df.get_trend_rows(group_feat = 'driver_race',trend_type ='lin_reg' )

60  total rows meet the criteria


,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,subgroup_trend2,subgroup_trend_strength2,subgroup2,distance
3632,num_stops,search_conducted_rate,0.00825551,51.319235,driver_race,Asian,lin_reg,0.0111067,38.412873,Black,0.001815
3633,num_stops,search_conducted_rate,0.00825551,51.319235,driver_race,Asian,lin_reg,0.0317734,1167.148168,Hispanic,0.014965
3634,num_stops,search_conducted_rate,0.00825551,51.319235,driver_race,Asian,lin_reg,0.00508864,62.781159,Other,0.002016
3635,num_stops,search_conducted_rate,0.00825551,51.319235,driver_race,Asian,lin_reg,0.0131983,1090.504695,White,0.003146
3636,num_stops,search_conducted_rate,0.0111067,38.412873,driver_race,Black,lin_reg,0.0317734,1167.148168,Hispanic,0.013150
3637,num_stops,search_conducted_rate,0.0111067,38.412873,driver_race,Black,lin_reg,0.00508864,62.781159,Other,0.003831
3638,num_stops,search_conducted_rate,0.0111067,38.412873,driver_race,Black,lin_reg,0.0131983,1090.504695,White,0.001331
3639,num_stops,search_conducted_rate,0.0317734,1167.148168,driver_race,Hispanic,lin_reg,0.00508864,62.781159,Other,0.016981
3640,num_stops,search_conducted_rate,0.0317734,1167.148168,driver_race,Hispanic,lin_reg,0.0131983,1090.504695,White,0.011819
3641,num_stops,search_conducted_rate,0.00508864,62.781159,driver_race,Other,lin_reg,0.0131983,1090.504695,White,0.005162


We can also filter based on SP detections with `

In [31]:
labeled_df.get_SP_rows(thresh=.2)

,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,subgroup_trend2,subgroup_trend_strength2,subgroup2,distance,SP_thresh0.2
1,contraband_found_rate,hit_rate,0.621241,0.621241,state,AZ,pearson_corr,-0.0367598,0.036760,CT,1.000000,True
12,contraband_found_rate,hit_rate,0.0128229,0.012823,state,CO,pearson_corr,-0.0367598,0.036760,CT,1.000000,True
23,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.380135,0.380135,IL,1.000000,True
24,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.405733,0.405733,MA,1.000000,True
25,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.148585,0.148585,MD,1.000000,True
26,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.277856,0.277856,NC,1.000000,True
27,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.130845,0.130845,RI,1.000000,True
28,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.122165,0.122165,SC,1.000000,True
29,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.313004,0.313004,TX,1.000000,True
30,contraband_found_rate,hit_rate,-0.0367598,0.036760,state,CT,pearson_corr,0.0888378,0.088838,VT,1.000000,True


## Detection

Detection via `get_SP_rows` happens in two steps:
1. label the rows
2. filter by that column to return

Labeling the rows can happen in a number of ways too, the detection accepts a number of forms of input, custom detections can be built in many ways

In [32]:
help(labeled_df.label_SP_rows)

Help on method label_SP_rows in module wiggum.ranking_processing:

label_SP_rows(filter_thresh=None) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    update the result_df with an additional colulmn indicateing rows with SP
    (or SP-like) as defined by sp_type
    
    Parameters
    -----------
    
    self : LabeledDataFrame
        must have values in result_df
    filter_thresh : dict or string
        dictionary of column label, threshold pairs or string name of a
        prespecified dictionary if dict, must include 'name' field (which
        will be used as the column name for storing the detections)



when filter_thresh is a dictionary, the filtering happens by taking the intersection of each row by the treshold prvided.  Some defaults are also built in accessible by string.

In [33]:
wg.trend_quality_sp

{'distance': 0.2,
 'agg_trend_strength': 0.15,
 'subgroup_trend_strength': 0.15,
 'name': 'default_qual_sp'}

Which can be applied with:

In [34]:
labeled_df.get_SP_rows('default_qual_sp')

KeyError: ('agg_trend_strength', 'occurred at index 0')

In [ ]:
wg.DEFAULT_SP_DEF

Which can be applied with:

In [ ]:
labeled_df.get_SP_rows('SP')

We can also define our own detection filters, using any available column

In [ ]:
lin_only_qual = {'name':'lin_only_qual_sp','distance':.2, 'agg_trend_strength':.05,
                'subgroup_trend_strength':.15,'trend_type':'lin_reg'}
labeled_df.get_SP_rows(lin_only_qual,replace=True) 

# Ranking

In [ ]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

In [ ]:
labeled_df.add_view_score('SP_thresh0.2',agg_type='sum',colored=False).head(10)

In [ ]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh0.2','SP_thresh0.2').head()